## Wolf Survey: Data Harmonization & License Merge (Python)

## Purpose:
- Read 4 SPSS survey files
- Harmonize differing schemas into one table
- Standardize a join key (common_address) and merge hunting/fishing license flags
- Export a clean CSV for downstream analysis & plotting

## Why this matters

Real survey pipelines involve messy schemas and external merges. This notebook demonstrates a real-life expample of such a task

## Tech Stack

pandas, pyreadstat, numpy

# 1. Import and Setup

In [9]:
import sys, os, re, glob
import pandas as pd
import numpy as np

# 2. Read survey .sav files and tag with group variable

###### I infer the stakeholder group from the filename (Deer / Land / House / Wolf) and add a Group column.

In [10]:
import pyreadstat as prs

# data directory
DATA_DIR = r"D:\MontanaBackup\WolfData\TrackingData"

files = [
    "Final Tracking Deer Elk Hunter Wolf Survey 2023.sav",
    "Final Tracking Landowner Wolf Survey 2023.sav",
    "Final Tracking Wolf Household Survey 2023.sav",
    "Final Tracking Wolf Hunter Wolf Survey 2023.sav",
]
paths = [os.path.join(DATA_DIR, f) for f in files]

def infer_group_from_name(fname: str) -> str:
    n = os.path.basename(fname).lower()
    if "deer" in n: return "Deer"
    if "land" in n: return "Land"
    if "house" in n: return "House"
    if "wolf" in n: return "Wolf"
    return "Unknown"

dfs = []
for p in paths:
    df, meta = prs.read_sav(p, apply_value_formats=False)
    df["Group"] = infer_group_from_name(p)
    dfs.append(df)

len(dfs), [d.shape for d in dfs], [d["Group"].iloc[0] for d in dfs]

(4,
 [(1500, 25), (2487, 21), (5000, 22), (1000, 24)],
 ['Deer', 'Land', 'House', 'Wolf'])

# 3. Harmonize across surveys

###### Each file can have slightly different columns. I create the superset of columns and add any missing as NaN. I also prefer ADDRESS (renaming ADDRESS1 → ADDRESS when needed) so downstream address logic has a consistent key.

In [27]:
all_cols = list({c for d in dfs for c in d.columns})

harmonized = []
for d in dfs:
    missing = [c for c in all_cols if c not in d.columns]
    for c in missing:
        d[c] = np.nan

    if "ADDRESS1" in d.columns and "ADDRESS" not in d.columns:
        d = d.rename(columns={"ADDRESS1": "ADDRESS"})

    d = d[all_cols]
    harmonized.append(d)

merged = pd.concat(harmonized, ignore_index=True)
merged.shape

(9987, 49)

In [15]:
merged = pd.concat(harmonized, ignore_index=True)
merged.shape, merged["Group"].value_counts(dropna=False)

((9987, 49),
 Group
 House    5000
 Land     2487
 Deer     1500
 Wolf     1000
 Name: count, dtype: int64)

# 4. Clean ZIP + build COMMMON_ADDRESS

###### I standardize ZIP → 5 digits, concatenate ADDRESS (optionally with ADDRESS2 if it starts with APT/UNIT/TRLR), CITY, STATE, and ZIP. All uppercase, single spaces. This gives an exact-match join key across sources.

In [28]:
def zip5(z):
    if pd.isna(z): return np.nan
    s = re.sub(r"[^0-9]", "", str(z))
    return s[:5] if s else np.nan

merged["ZIP"] = merged["ZIP"].map(zip5) if "ZIP" in merged.columns else np.nan

def build_common_address(row):
    addr = row.get("ADDRESS")
    addr2 = row.get("ADDRESS2")
    if pd.notna(addr2) and re.match(r"^(APT|UNIT|TRLR)", str(addr2), flags=re.I):
        addr = f"{addr} {addr2}" if pd.notna(addr) else addr2
    elif (pd.isna(addr) or str(addr).strip()=="") and pd.notna(addr2):
        addr = addr2
    parts = [addr, row.get("CITY"), row.get("STATE"), row.get("ZIP")]
    out = " ".join([str(p) for p in parts if pd.notna(p) and str(p).strip()])
    out = re.sub(r"\s+", " ", out).strip()
    return out.upper() if out else np.nan

merged["COMMON_ADDRESS"] = merged.apply(build_common_address, axis=1)
merged[["ADDRESS","ADDRESS2","CITY","STATE","ZIP","COMMON_ADDRESS"]].head()

,ADDRESS,ADDRESS2,CITY,STATE,ZIP,COMMON_ADDRESS
0,2 MICHAEL ST,NaN,GLASGOW,MT,59230,2 MICHAEL ST GLASGOW MT 59230
1,2 TIMBER RIDGE CT,NaN,CLANCY,MT,59634,2 TIMBER RIDGE CT CLANCY MT 59634
2,70 JOHNNYS COAL RD,NaN,ROUNDUP,MT,59072,70 JOHNNYS COAL RD ROUNDUP MT 59072
3,1 JACKSON CREEK RD PMB 2005,NaN,CLANCY,MT,59634,1 JACKSON CREEK RD PMB 2005 CLANCY MT 59634
4,1 MERGENTHALER RD,NaN,CLANCY,MT,59634,1 MERGENTHALER RD CLANCY MT 59634


# 5. Add hunting/fishing license flags

###### I read the 2023 hunting/fishing license lists and build the exact same COMMON_ADDRESS format so the join is apples-to-apples.

In [29]:
# paths to your license CSVs
hunt_path = os.path.join(DATA_DIR, "Copy of Resident Base Hunting List 2023.csv")
fish_path = os.path.join(DATA_DIR, "Copy of Resident Fishing List 2023.csv")

hunt = pd.read_csv(hunt_path, dtype={hunt.columns[1]: str})
fish = pd.read_csv(fish_path)

def prep_license(df):
    # try to find columns by common names
    def first(*cands):
        for c in cands:
            if c in df.columns: return c
    addr = first("Address","ADDRESS","address")
    city = first("City","CITY","city")
    state= first("State","STATE","state")
    zipc = first("Zip","ZIP","zip")

    df["Zip5"] = df[zipc].map(zip5)
    parts = []
    for c in [addr, city, state, "Zip5"]:
        parts.append(df[c].astype(str) if c in df.columns else "")
    addr_str = (parts[0].str.cat(parts[1], sep=" ")
                          .str.cat(parts[2], sep=" ")
                          .str.cat(parts[3], sep=" "))
    addr_str = addr_str.str.replace(r"\s+", " ", regex=True).str.strip().str.upper()
    return addr_str.dropna().unique()

hunt_addrs = set(prep_license(hunt))
fish_addrs = set(prep_license(fish))

merged["hunting_license"] = np.where(merged["COMMON_ADDRESS"].isin(hunt_addrs), 1, 0)
merged["fishing_license"] = np.where(merged["COMMON_ADDRESS"].isin(fish_addrs), 1, 0)

merged[["COMMON_ADDRESS","hunting_license","fishing_license"]].head()

,COMMON_ADDRESS,hunting_license,fishing_license
0,2 MICHAEL ST GLASGOW MT 59230,1,1
1,2 TIMBER RIDGE CT CLANCY MT 59634,0,0
2,70 JOHNNYS COAL RD ROUNDUP MT 59072,0,0
3,1 JACKSON CREEK RD PMB 2005 CLANCY MT 59634,0,0
4,1 MERGENTHALER RD CLANCY MT 59634,0,0


# 6. Export clean dataset

In [30]:
OUT_PATH = os.path.join(DATA_DIR, "Wolf_Survey_Cleaned.csv")
merged.to_csv(OUT_PATH, index=False)
OUT_PATH

'D:\\MontanaBackup\\WolfData\\TrackingData\\Wolf_Survey_Cleaned.csv'